In [ ]:
import os
import argparse
import cv2
import numpy as np
import sys
import time
from threading import Thread
import importlib.util
from PIL import Image
import math
use_TPU = False
# # Import TensorFlow libraries
# # If tensorflow is not installed, import interpreter from tflite_runtime, else import from regular tensorflow
# # If using Coral Edge TPU, import the load_delegate library
# pkg = importlib.util.find_spec('tensorflow')
# if pkg is None:
#     from tflite_runtime.interpreter import Interpreter
#     if use_TPU:
#         from tflite_runtime.interpreter import load_delegate
# else:
# #     from tensorflow.lite.python.interpreter import Interpreter
#     if use_TPU:
#         from tensorflow.lite.python.interpreter import load_delegate

# # If using Edge TPU, assign filename for Edge TPU model
# if use_TPU:
#     # If user has specified the name of the .tflite file, use that name, otherwise use default 'edgetpu.tflite'
#     if (GRAPH_NAME == 'detect.tflite'):
#         GRAPH_NAME = 'edgetpu.tflite' 

In [ ]:
from tensorflow.lite.python.interpreter import Interpreter

In [ ]:
class VideoStream:
    """Camera object that controls video streaming from the Picamera"""
    def __init__(self,resolution=(640,480),framerate=30):
        # Initialize the PiCamera and the camera image stream
        self.stream = cv2.VideoCapture(0)
        ret = self.stream.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
        ret = self.stream.set(3,resolution[0])
        ret = self.stream.set(4,resolution[1])
            
        # Read first frame from the stream
        (self.grabbed, self.frame) = self.stream.read()

	# Variable to control when the camera is stopped
        self.stopped = False

    def start(self):
	# Start the thread that reads frames from the video stream
        Thread(target=self.update,args=()).start()
        return self

    def update(self):
        # Keep looping indefinitely until the thread is stopped
        while True:
            # If the camera is stopped, stop the thread
            if self.stopped:
                # Close camera resources
                self.stream.release()
                return

            # Otherwise, grab the next frame from the stream
            (self.grabbed, self.frame) = self.stream.read()

    def read(self):
	# Return the most recent frame
        return self.frame

    def stop(self):
	# Indicate that the camera and thread should be stopped
        self.stopped = True

In [ ]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def argmax2d(inp_3d):
    """
    Get the x,y positions of the heatmap of each part's argmax()
    """
    heatmapPositions = np.zeros(shape=(17,2))
    heatmapConf = np.zeros(shape=(17,1))
    for i in range(17):
        argmax_i =  np.unravel_index(inp_3d[:,:,i].argmax(), inp_3d[:,:,i].shape)
        max_i =  inp_3d[:,:,i].max()
        heatmapPositions[i,:] = argmax_i
        heatmapConf[i,:] = max_i
    return heatmapPositions,heatmapConf

def get_offsetVector(heatmapPositions=None,offsets=None):
    allArrays = np.zeros(shape=(17,2))
    for idx,el in enumerate(heatmapPositions):
#         print(el)
        allArrays[idx,0] = offsets[int(el[0]),int(el[1]),idx]
        allArrays[idx,1] = offsets[int(el[0]),int(el[1]),17+idx]
    return allArrays

def draw_arrows(frame):
    """Show the direction vector output in the cv2 window"""
    #cv2.putText(frame,"Color:", (0, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, 255, thickness=2)
    cv2.arrowedLine(frame, (int(imW/2), int(imH/2)),
                    (int(imW/2) + xoff, int(imH/2) - yoff),
                    (0, 0, 255), 1)
    return frame
    
MODEL_NAME = "Sample_TFLite_model"
MODEL_NAME = "pose_TFLite_model"
GRAPH_NAME = 'detect.tflite'
LABELMAP_NAME = 'labelmap.txt'
min_conf_threshold = float(0.5)
resW, resH = '1280x720'.split('x')
imW, imH = int(resW), int(resH)
min_thresh = 0.7

In [ ]:
# print in terminal
import sys
sys.stdout = open('/dev/stdout', 'w')

In [ ]:
# Get path to current working directory
CWD_PATH = os.getcwd()

# Path to .tflite file, which contains the model that is used for object detection
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,GRAPH_NAME)

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,MODEL_NAME,LABELMAP_NAME)

# Load the label map
with open(PATH_TO_LABELS, 'r') as f:
    labels = [line.strip() for line in f.readlines()]

# Have to do a weird fix for label map if using the COCO "starter model" from
# https://www.tensorflow.org/lite/models/object_detection/overview
# First label is '???', which has to be removed.
if labels[0] == '???':
    del(labels[0])

# Load the Tensorflow Lite model.
# If using Edge TPU, use special load_delegate argument
if use_TPU:
    interpreter = Interpreter(model_path=PATH_TO_CKPT,
                              experimental_delegates=[load_delegate('libedgetpu.so.1.0')])
    print(PATH_TO_CKPT)
else:
    interpreter = Interpreter(model_path=PATH_TO_CKPT)

interpreter.allocate_tensors()
interpreter.invoke()
# Get model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

floating_model = (input_details[0]['dtype'] == np.float32)

input_mean = width/2
input_std = width/2

# Initialize frame rate calculation
frame_rate_calc = 1
freq = cv2.getTickFrequency()

xoff, yoff = 0,0
# Initialize video stream
videostream = VideoStream(resolution=(imW,imH),framerate=30).start()
time.sleep(1)
#for frame1 in camera.capture_continuous(rawCapture, format="bgr",use_video_port=True):
while True:

    # Start timer (for calculating frame rate)
    t1 = cv2.getTickCount()
    
    # Grab frame from video stream
    frame1 = videostream.read()

    # Acquire frame and resize to expected shape [1xHxWx3]
    frame = frame1.copy()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, (width, height))
    input_data = np.expand_dims(frame_resized, axis=0)
    # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
    if floating_model:
        input_data = (np.float32(input_data) - input_mean) / input_std
    # Perform the actual detection by running the model with the image as input

    interpreter.set_tensor(input_details[0]['index'],input_data)
    interpreter.invoke()
    heatmapscores = interpreter.get_tensor(output_details[0]['index'])[0] # Bounding box coordinates of detected objects
    offsets = interpreter.get_tensor(output_details[1]['index'])[0] # Class index of detected objects
    # displacementFwd = interpreter.get_tensor(output_details[2]['index'])[0]
    # displacementBwd = interpreter.get_tensor(output_details[3]['index'])[0]
    
    # define vectorized sigmoid
    sigmoid_v = np.vectorize(sigmoid)
    # 1 sigmoid
    sigmoheatmapscores = sigmoid_v(heatmapscores)
    # 2 argmax2d
    heatmapPositions,heatmapConfidence = argmax2d(sigmoheatmapscores)
    # 3 offsetVectors
    offsetVectors = get_offsetVector(heatmapPositions,offsets)
    # 4 keypointPositions
    outputStride = 32
    keypointPositions = heatmapPositions * outputStride + offsetVectors
    # 5 draw keypoints
    for idx,el in enumerate(heatmapConfidence):
        if heatmapConfidence[idx][0] >= min_thresh:
            x = round((keypointPositions[idx][1]/width)*imW)
            y = round((keypointPositions[idx][0]/height)*imH)
            if 'right' in labels[idx]:
                cv2.circle(frame,(int(x),int(y)), 5, (0,255,0), -1)
            elif 'left' in labels[idx]:
                cv2.circle(frame,(int(x),int(y)), 5, (0,0,255), -1)
            else:
                xoff, yoff = int(x-(imW/2)),int((imH/2)-y)
                cv2.circle(frame,(int(x),int(y)), 5, (255,0,0), -1)
    draw_arrows(frame)
    # Draw framerate in corner of frame
    cv2.putText(frame,
                'FPS: {0:.2f}'.format(frame_rate_calc),
                (imW-200,30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255,255,0),
                1,
                cv2.LINE_AA)

    
    distance = 100
    cmd = ""
    # print(yoff)
    tracking = True
    if tracking:
        if xoff < -distance and xoff>-imW/2:
            cmd = "counter_clockwise"
        elif xoff > distance and xoff<imW/2:
            cmd = "clockwise"
        elif yoff < -distance and yoff>-imH/2:
            print("DOWNNNNN",yoff)
            cmd = "down"
        elif yoff > distance and yoff<imH/2:
            print("UPPPPPPPPPPPPPPP",yoff)
            # cmd = "up"
        elif xoff==0 and yoff == 0:
            print("ignore")
    
    # All the results have been drawn on the frame, so it's time to display it.
    cv2.imshow('Object detector', frame)
                    
    # Calculate framerate
    t2 = cv2.getTickCount()
    time1 = (t2-t1)/freq
    frame_rate_calc= 1/time1

    # Press 'q' to quit
    if cv2.waitKey(1) == ord('q'):
        break